# exc 1a

Creating a dataset for:

$$
f(x) = \frac{1}{1+25x^2}, \quad x \in [-1,1]
$$

In [1]:
#import packages
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x = np.linspace(-1,1).reshape(-1,1) #x within interval [-1,1]
denominator = 1+(25*x**2)
#defining y and adding noise as a normal distrubution
y = 1/denominator + np.random.normal(0,1, x.shape) 
